<a href="https://colab.research.google.com/github/Ugo5738/Web-crawlers/blob/main/Link_Crawler_for_Guardian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scrapy

In [ ]:
import scrapy
import pandas as pd

In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/

/content/drive/My Drive/Colab Notebooks


In [ ]:
!scrapy startproject Link_Crawler

Error: scrapy.cfg already exists in /content/drive/My Drive/Colab Notebooks/Link_Crawler


# 1. Link Crawler For Guardian

In [ ]:
# Create a python file inside the /content/drive/MyDrive/Colab Notebooks/crawler_tutorial/crawler_tutorial/spiders folder and put this code in there

import scrapy

class WebSpider(scrapy.Spider):
    name = "guardian_links"

    start_urls = [ 
                   "https://guardian.ng/category/technology/"
                   ]
    
    # Select the technology subcategories from the drop-down and loop through them
    def parse(self, response):
        subcategories = response.css("div.dropdown-menu#menu-11173 a::attr(href)").extract()
        for subcategory in subcategories:
            yield scrapy.Request(url=subcategory, callback=self.parse_subcategory)
        
    # In each subcategory, extract the links in the first page
    def parse_subcategory(self, response):
        first_category_table = response.css("div.category-table")[0]
        links = first_category_table.css("a::attr(href)").extract()
        for link in links:
            yield scrapy.Request(url=link, callback=self.parse_article)

        # Use pagination to extract the links in the next pages
        next_page_url = response.xpath("//a[@class='next-page']/@href").extract_first()
        if next_page_url:
            yield scrapy.Request(url=next_page_url, callback=self.parse_subcategory)

    def parse_article(self, response):
        link = response.url
        datetime = response.css("div.manual-age.single-article-datetime::text").extract()[0]
        first = datetime.replace('\t', '')
        sec = first.replace('\n', '')
        third = sec.split(' ')[0:3]
        date = [third[1], third[0], third[2]]
        yield {
            "links": link
            "dates": date
        }

In [ ]:
import scrapy

class WebSpider(scrapy.Spider):
    name = "guardian_links"

    start_urls = [ 
                   "https://guardian.ng/category/technology/"
                   ]
    
    # Select the technology subcategories from the drop-down and loop through them
    def parse(self, response):
        subcategories = response.css("div.dropdown-menu#menu-11173 a::attr(href)").extract()
        for subcategory in subcategories:
            yield scrapy.Request(url=subcategory, callback=self.parse_subcategory)
        
    # In each subcategory, extract the links in the first page
    def parse_subcategory(self, response):
        first_category_table = response.css("div.category-table")[0]
        yield {
            "links":first_category_table.css("a::attr(href)").extract()
          }
        # Use pagination to extract the links in the next pages
        next_page_url = response.xpath("//a[@class='next-page']/@href").extract_first()
        if next_page_url:
            yield scrapy.Request(url=next_page_url, callback=self.parse_subcategory)

In [ ]:
# To view all the crawled links

%cd /content/drive/MyDrive/Colab Notebooks/Link_Crawler

!scrapy crawl guardian_links

In [ ]:
# For test and interaction

!scrapy shell "https://guardian.ng/technology/fuge-ecommerce-platform-unveils-new-service/"

2020-12-23 20:18:19 [scrapy.crawler] INFO: Received SIGTERM, shutting down gracefully. Send again to force 
^C


In [ ]:
# To append the crawled links to the Guardian links txt file

%cd /content/drive/MyDrive/Colab Notebooks/Link_Crawler

!scrapy crawl guardian_links -o guardian_links.csv

In [ ]:
import pandas as pd
df1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Link_Crawler/guardian_links.csv") 
df1.head()

,links
0,https://guardian.ng/technology/agency-to-drive...
1,https://guardian.ng/technology/mtn-trains-2000...
2,https://guardian.ng/technology/coronavirus-joe...
3,https://guardian.ng/technology/5g-will-generat...
4,https://guardian.ng/technology/technology-tech...


In [ ]:
links = []
for link in df1['links']:
    links_split = link.split(',')
    for url in links_split:
        links.append(url)
print(len(links))
# but appended only 3652 articles

5007


In [ ]:
!pip install newspaper3k

from newspaper import Article
import nltk
nltk.download('punkt')

In [ ]:
# build a function to convert keywords list to strings

seperator = ', '

def converttostr(input_seq, seperator):
    final_str = seperator.join(input_seq)
    return final_str

for ind, url in enumerate(links[:3000], start=1):
    article = Article(url, language="en")
    article.download() 
    article.parse() 
    article.nlp() 

    with open("corpus.txt", "a") as f:
        keywords = converttostr(article.keywords, seperator)
        f.write(f"Keywords: [{keywords}]" + "\n" + "\n")
        f.write(article.title + "\n" + "\n")
        f.write(article.text + "\n" + "\n")
        f.write(f"This is the article summary \n{article.summary}" + "\n" + "\n" + "\n" + "\n" + "\n")

    print(f"Finish with article {ind}")

Finish with article 1
Finish with article 2
Finish with article 3
Finish with article 4
Finish with article 5
Finish with article 6
Finish with article 7
Finish with article 8
Finish with article 9
Finish with article 10
Finish with article 11
Finish with article 12
Finish with article 13
Finish with article 14
Finish with article 15
Finish with article 16
Finish with article 17
Finish with article 18
Finish with article 19
Finish with article 20
Finish with article 21
Finish with article 22
Finish with article 23
Finish with article 24
Finish with article 25
Finish with article 26
Finish with article 27
Finish with article 28
Finish with article 29
Finish with article 30
Finish with article 31
Finish with article 32
Finish with article 33
Finish with article 34
Finish with article 35
Finish with article 36
Finish with article 37
Finish with article 38
Finish with article 39
Finish with article 40
Finish with article 41
Finish with article 42
Finish with article 43
Finish with article 

# 2. Link Crawler for Technopreneur Startups

In [ ]:
# Create a python file inside the /content/drive/MyDrive/Colab Notebooks/crawler_tutorial/crawler_tutorial/spiders folder and put this code in there

import scrapy

class WebSpider(scrapy.Spider):
    name = "technopreneur_links"

    start_urls = [ 
                  "https://technopreneur.com.ng/category/news/",
                  "https://technopreneur.com.ng/category/startups/"
                   ]
    
    # Extract the links in the first page
    def parse(self, response):
        articles = response.css("h3.article-title")
        yield {
            "links":articles.css("a::attr(href)").extract()
          }
        # Use pagination to extract the links in the next pages
        next_page_url = response.xpath("//span[@class='last-page first-last-pages']/a/@href").extract_first()
        if next_page_url:
            yield scrapy.Request(url=next_page_url, callback=self.parse)

In [ ]:
# To view all the crawled links

%cd /content/drive/MyDrive/Colab Notebooks/Link_Crawler

!scrapy crawl technopreneur_links

In [ ]:
# For test and interaction

!scrapy shell "https://technopreneur.com.ng"

2020-12-21 19:31:28 [scrapy.crawler] INFO: Received SIGTERM, shutting down gracefully. Send again to force 
^C


In [ ]:
# To append the crawled links to the Guardian links txt file

%cd /content/drive/MyDrive/Colab Notebooks/Link_Crawler

!scrapy crawl technopreneur_links -o technopreneur_links.csv

In [ ]:
import pandas as pd
df2 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Link_Crawler/technopreneur_links.csv") 
df2.head()

,links
0,https://technopreneur.com.ng/2020/11/06/ventur...
1,https://technopreneur.com.ng/2020/11/06/ventur...
2,https://technopreneur.com.ng/2020/06/10/alibab...
3,https://technopreneur.com.ng/2019/12/13/shuttl...
4,https://technopreneur.com.ng/2020/03/28/covid-...


In [ ]:
links = []
for link in df2['links']:
    links_split = link.split(',')
    for url in links_split:
        links.append(url)
print(len(links))

658


In [ ]:
!pip install newspaper3k

from newspaper import Article
import nltk
nltk.download('punkt')

In [ ]:
# build a function to convert keywords list to strings

seperator = ', '

def converttostr(input_seq, seperator):
    final_str = seperator.join(input_seq)
    return final_str

for ind, url in enumerate(links, start=1):
    article = Article(url, language="en")
    article.download() 
    article.parse() 
    article.nlp() 

    
    with open("corpus.txt", "a") as f:
        keywords = converttostr(article.keywords, seperator)
        f.write(f"Keywords: [{keywords}]" + "\n" + "\n")
        f.write(article.title + "\n")
        f.write(date + "\n" + "\n")
        f.write(article.text + "\n" + "\n")
        f.write(f"This is the article summary \n{article.summary}" + "\n" + "\n" + "\n" + "\n" + "\n")

    print(f"Finish with article {ind}")

Finish with article 1
Finish with article 2
Finish with article 3
Finish with article 4
Finish with article 5
Finish with article 6
Finish with article 7
Finish with article 8
Finish with article 9
Finish with article 10
Finish with article 11
Finish with article 12
Finish with article 13
Finish with article 14
Finish with article 15
Finish with article 16
Finish with article 17
Finish with article 18
Finish with article 19
Finish with article 20
Finish with article 21
Finish with article 22
Finish with article 23
Finish with article 24
Finish with article 25
Finish with article 26
Finish with article 27
Finish with article 28
Finish with article 29
Finish with article 30
Finish with article 31
Finish with article 32
Finish with article 33
Finish with article 34
Finish with article 35
Finish with article 36
Finish with article 37
Finish with article 38
Finish with article 39
Finish with article 40
Finish with article 41
Finish with article 42
Finish with article 43
Finish with article 

# 3. Link Crawler For Techpoint

In [ ]:
# Create a python file inside the /content/drive/MyDrive/Colab Notebooks/crawler_tutorial/crawler_tutorial/spiders folder and put this code in there

import scrapy

class WebSpider(scrapy.Spider):
    name = "techpoint_scrape"

    start_urls = ["https://techpoint.africa/subject/startups/"]

    # Get the links in a particular page
    def parse(self, response):
        links = response.css("h5.entry-title a::attr(href)").extract()[:-1]
        for link in links:
            datetime_inv = link.split("/")[3:6]
            datetime = [datetime_inv[1], datetime_inv[2], datetime_inv[0]]
            date = ' '.join(datetime)
            yield {
              "links":link,
              "date":date
              }
        # Use pagination to extract the links in the next pages
        next_page_url = response.css("a.next.page-numbers ::attr(href)").extract_first()
        if next_page_url:
            yield scrapy.Request(url=next_page_url, callback=self.parse)

In [ ]:
# To view all the crawled links

%cd /content/drive/MyDrive/Colab Notebooks/Link_Crawler

!scrapy crawl techpoint_scrape

In [ ]:
# For test and interaction

!scrapy shell "https://techpoint.africa/subject/startups/"

In [ ]:
link = response.css("h5.entry-title a::attr(href)").extract_first()
datetime_inv = link.split("/")[3:6]
datetime = [datetime_inv[1], datetime_inv[2], datetime_inv[0]]
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
months = ["Jan.", "Feb.", "Mar.", "Apr.", "May", "June", "July", "Aug.", "Sep.", "Oct.", "Nov.", "Dec."]

for ind, item in enumerate(numbers):
    if numbers[ind] == datetime[0]: datetime[0] = months[item]

date = [str(i) for i in datetime]
date = ' '.join(date)
print(date)

In [ ]:
# To append the crawled links to the Guardian links txt file

%cd /content/drive/MyDrive/Colab Notebooks/Link_Crawler

!scrapy crawl techpoint_scrape -o techpoint_scrape.csv

In [ ]:
import pandas as pd
# use pandas to input the keys and values on every iteration
df3 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Link_Crawler/techpoint_scrape.csv") 
df3.head()

,links,date
0,https://techpoint.africa/2020/12/22/what-54gen...,12 22 2020
1,https://techpoint.africa/2020/12/21/damilola-o...,12 21 2020
2,https://techpoint.africa/2020/12/21/kiakiaprin...,12 21 2020
3,https://techpoint.africa/2020/12/16/onepipe-ra...,12 16 2020
4,https://techpoint.africa/2020/12/15/ilara-heal...,12 15 2020


In [ ]:
# Remove duplicates
links = list(set(df3['links'].values.tolist()))
dates = df3['date'].values.tolist()

In [ ]:
print(f"The number of links is {len(links)}")
print(f"The number of links is {len(dates)}")

The number of links is 733
The number of links is 733


In [ ]:
!pip install newspaper3k

from newspaper import Article
import nltk
nltk.download('punkt')

In [ ]:
# build a function to convert keywords list to strings

seperator = ', '

def converttostr(input_seq, seperator):
    final_str = seperator.join(input_seq)
    return final_str

count = 1
# list_of_urls should contain two columns: The link, date-time
for url, date in zip(links, dates):
    article = Article(url, language="en")
    article.download() 
    article.parse() 
    article.nlp() 

    
    with open("corpus.txt", "a") as f:
        keywords = converttostr(article.keywords, seperator)
        f.write(f"Keywords: [{keywords}]" + "\n" + "\n")
        f.write(article.title + "\n")
        f.write(date + "\n" + "\n")
        f.write(article.text + "\n" + "\n")
        f.write(f"This is the article summary \n{article.summary}" + "\n" + "\n" + "\n" + "\n" + "\n")
        
    print(f"Finish with article {count}")
    count += 1

Finish with article 1
Finish with article 2
Finish with article 3
Finish with article 4
Finish with article 5
Finish with article 6
Finish with article 7
Finish with article 8
Finish with article 9
Finish with article 10
Finish with article 11
Finish with article 12
Finish with article 13
Finish with article 14
Finish with article 15
Finish with article 16
Finish with article 17
Finish with article 18
Finish with article 19
Finish with article 20
Finish with article 21
Finish with article 22
Finish with article 23
Finish with article 24
Finish with article 25
Finish with article 26
Finish with article 27
Finish with article 28
Finish with article 29
Finish with article 30
Finish with article 31
Finish with article 32
Finish with article 33
Finish with article 34
Finish with article 35
Finish with article 36
Finish with article 37
Finish with article 38
Finish with article 39
Finish with article 40
Finish with article 41
Finish with article 42
Finish with article 43
Finish with article 

# 4. Link Crawler For AppleInsider

In [ ]:
# Create a python file inside the /content/drive/MyDrive/Colab Notebooks/crawler_tutorial/crawler_tutorial/spiders folder and put this code in there

import scrapy

class WebSpider(scrapy.Spider):
    name = "appleinsider_scrape"

    start_urls = ["https://appleinsider.com/"]

    # Get the links in a particular page
    def parse(self, response):
        links = response.css("h2.blueHover a::attr(href)").extract()[0:40]
        for link in links:
            datetime_inv = link.split("/")[2:4]
            datetime = [datetime_inv[1], datetime_inv[2], datetime_inv[0]]
            date = ' '.join(datetime)
            yield {
              "links":link,
              "date":date
              }

        # Use pagination to extract the links in the next pages
        next_page_url = response.css("div#load-more a::attr(href)").extract_first()
        if next_page_url:
            next_page_url = response.urljoin(next_page_url)
            yield scrapy.Request(url=next_page_url, callback=self.parse)

In [ ]:
# To view all the crawled links

%cd /content/drive/MyDrive/Colab Notebooks/Link_Crawler

!scrapy crawl appleinsider_scrape

In [ ]:
# For test and interaction

!scrapy shell "https://appleinsider.com/"

2020-12-22 17:22:53 [scrapy.crawler] INFO: Received SIGTERM, shutting down gracefully. Send again to force 
^C


In [ ]:
link = response.css("h5.entry-title a::attr(href)").extract_first()
datetime_inv = link.split("/")[3:6]
datetime = [datetime_inv[1], datetime_inv[2], datetime_inv[0]]
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
months = ["Jan.", "Feb.", "Mar.", "Apr.", "May", "June", "July", "Aug.", "Sep.", "Oct.", "Nov.", "Dec."]

for ind, item in enumerate(numbers):
    if numbers[ind] == datetime[0]: datetime[0] = months[item]

date = [str(i) for i in datetime]
date = ' '.join(date)
print(date)

In [ ]:
# To append the crawled links to the Guardian links txt file

%cd /content/drive/MyDrive/Colab Notebooks/Link_Crawler

!scrapy crawl appleinsider_scrape -o appleinsider_scrape2.csv

In [ ]:
import pandas as pd
# use pandas to input the keys and values on every iteration
df4 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Link_Crawler/appleinsider_scrape2.csv") 
df4.head()

,links,date
0,https://appleinsider.com/articles/20/12/25/new...,12 25 20
1,https://appleinsider.com/articles/20/02/01/app...,02 01 20
2,https://appleinsider.com/articles/20/12/25/fir...,12 25 20
3,https://appleinsider.com/articles/20/12/25/app...,12 25 20
4,https://appleinsider.com/articles/20/12/25/yea...,12 25 20


In [ ]:
# Remove duplicates
links = df4['links'].values.tolist()
dates = df4['date'].values.tolist()

In [ ]:
print(f"The number of links is {len(links)}")
print(f"The number of dates is {len(dates)}")

The number of links is 30800
The number of dates is 30800


In [ ]:
print(links[1999])

https://appleinsider.com/articles/20/09/01/first-long-way-up-trailer-with-ewan-mcgregor-hints-at-tricky-13000-mile-journey


In [ ]:
!pip install newspaper3k

from newspaper import Article
import nltk
nltk.download('punkt')

In [ ]:
# build a function to convert keywords list to strings

seperator = ', '

def converttostr(input_seq, seperator):
    final_str = seperator.join(input_seq)
    return final_str

count = 9945
# list_of_urls should contain two columns: The link, date-time
for url, date in zip(links[9945:10000], dates):
    article = Article(url, language="en")
    article.download() 
    article.parse() 
    article.nlp() 
    
    with open("corpus.txt", "a") as f:
        keywords = converttostr(article.keywords, seperator)
        f.write(f"Keywords: [{keywords}]" + "\n" + "\n")
        f.write(article.title + "\n")
        f.write(date + "\n" + "\n")
        f.write(article.text + "\n" + "\n")
        f.write(f"This is the article summary \n{article.summary}" + "\n" + "\n" + "\n" + "\n" + "\n")
        
    print(f"Finish with article {count}")
    count += 1

Finish with article 9945
Finish with article 9946
Finish with article 9947
Finish with article 9948
Finish with article 9949
Finish with article 9950
Finish with article 9951
Finish with article 9952
Finish with article 9953
Finish with article 9954
Finish with article 9955
Finish with article 9956
Finish with article 9957
Finish with article 9958
Finish with article 9959
Finish with article 9960
Finish with article 9961
Finish with article 9962
Finish with article 9963
Finish with article 9964
Finish with article 9965
Finish with article 9966
Finish with article 9967
Finish with article 9968
Finish with article 9969
Finish with article 9970
Finish with article 9971
Finish with article 9972
Finish with article 9973
Finish with article 9974
Finish with article 9975
Finish with article 9976
Finish with article 9977
Finish with article 9978
Finish with article 9979
Finish with article 9980
Finish with article 9981
Finish with article 9982
Finish with article 9983
Finish with article 9984


# 5. Link Crawler For Techradar

In [ ]:
# Create a python file inside the /content/drive/MyDrive/Colab Notebooks/crawler_tutorial/crawler_tutorial/spiders folder and put this code in there

import scrapy

class WebSpider(scrapy.Spider):
    name = "techradar_scrape"

    start_urls = ["https://www.techradar.com/news/archive/2020/12", 
                  "https://www.techradar.com/news/archive/2020/11",
                  "https://www.techradar.com/news/archive/2020/10"]

    # Get the links in a particular page
    def parse(self, response):
        links = response.css("li.day-article a::attr(href)").extract()
        for link in links:
            yield scrapy.Request(url=link, callback=self.parse_dates)

        # Use pagination to extract the links in the next pages
        next_page_url = response.css("div#load-more a::attr(href)").extract_first()
        if next_page_url:
            next_page_url = response.urljoin(next_page_url)
            yield scrapy.Request(url=next_page_url, callback=self.parse)

    def parse_dates(self, response):
        page_url = response.url
        datetime = response.css("time.no-wrap.relative-date.chunk ::attr(datetime)").extract_first()
        if datetime is None:
            date = "There is no date here!"
        else:
            datetime = datetime.split('T')[0]
            datetime = datetime.split('-')
            date = [datetime[1], datetime[2], datetime[0]]

        yield {
            "links": page_url,
            "date": date
        }

In [ ]:
# To view all the crawled links

%cd /content/drive/MyDrive/Colab Notebooks/Link_Crawler

!scrapy crawl techradar_scrape

In [ ]:
# For test and interaction

!scrapy shell "https://www.techradar.com/news/why-the-low-code-industry-is-in-for-a-reckoning"

2020-12-23 14:37:55 [scrapy.crawler] INFO: Received SIGTERM, shutting down gracefully. Send again to force 
^C


In [ ]:
# To append the crawled links to the Guardian links txt file

%cd /content/drive/MyDrive/Colab Notebooks/Link_Crawler

!scrapy crawl techradar_scrape -o techradar_scrape.csv

In [ ]:
import pandas as pd
# use pandas to input the keys and values on every iteration
df5 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Link_Crawler/techradar_scrape.csv") 
df5.head()

,links
0,https://www.techradar.com/news/qnap-fixes-even...
1,https://www.techradar.com/news/zoom-is-about-t...
2,https://www.techradar.com/news/wordpress-wants...
3,https://www.techradar.com/news/best-domain-reg...
4,https://www.techradar.com/news/massive-next-ge...


In [ ]:
for index, row in df5.iterrows():
    if row['date'] == 'There is no date here!':
        df5.drop(index, inplace=True)

In [ ]:
# Remove duplicates
links = df5['links'].values.tolist()
# dates = df5['date'].values.tolist()

In [ ]:
print(f"The number of links is {len(links)}")
# print(f"The number of dates is {len(dates)}")

The number of links is 4092


In [ ]:
!pip install newspaper3k

from newspaper import Article
import nltk
nltk.download('punkt')

In [ ]:
# build a function to convert keywords list to strings

seperator = ', '

def converttostr(input_seq, seperator):
    final_str = seperator.join(input_seq)
    return final_str


# list_of_urls should contain two columns: The link, date-time
for ind, url in enumerate(links[2000:], start=2000):
    article = Article(url, language="en")
    article.download() 
    article.parse() 
    article.nlp() 
    
    with open("Technology File2.txt", "a") as f:
        keywords = converttostr(article.keywords, seperator)
        f.write(f"Keywords: [{keywords}]" + "\n" + "\n")
        f.write(article.title + "\n")
        f.write(date + "\n" + "\n")
        f.write(article.text + "\n" + "\n")
        f.write(f"This is the article summary \n{article.summary}" + "\n" + "\n" + "\n" + "\n" + "\n")

    print(f"Finish with article {ind}")

Finish with article 2000
Finish with article 2001
Finish with article 2002
Finish with article 2003
Finish with article 2004
Finish with article 2005
Finish with article 2006
Finish with article 2007
Finish with article 2008
Finish with article 2009
Finish with article 2010
Finish with article 2011
Finish with article 2012
Finish with article 2013
Finish with article 2014
Finish with article 2015
Finish with article 2016
Finish with article 2017
Finish with article 2018
Finish with article 2019
Finish with article 2020
Finish with article 2021
Finish with article 2022
Finish with article 2023
Finish with article 2024
Finish with article 2025
Finish with article 2026
Finish with article 2027
Finish with article 2028
Finish with article 2029
Finish with article 2030
Finish with article 2031
Finish with article 2032
Finish with article 2033
Finish with article 2034
Finish with article 2035
Finish with article 2036
Finish with article 2037
Finish with article 2038
Finish with article 2039


# 6. Link Crawler For The Verge

In [ ]:
# Create a python file inside the /content/drive/MyDrive/Colab Notebooks/crawler_tutorial/crawler_tutorial/spiders folder and put this code in there

import scrapy

class WebSpider(scrapy.Spider):
    name = "the_verge_scrape"

    start_urls = [
                  "https://www.theverge.com/archives/tech/2014/1",
                  ]

    # Get the links in a particular page
    def parse(self, response):
        year_links = response.css("li ::attr(data-slug)").extract()[:6]
        for year_link in year_links:
            yield scrapy.Request(url=month_link, callback=self.parse_month_links)
    
    def parse_month_links(self, response):
        month_links = response.css("li ::attr(data-slug)").extract()[10:]
        for month_link in month_links:
            yield scrapy.Request(url=month_link, callback=self.parse_page_links)

    def parse_page_links(self, response):
        page_links = response.css("h2.c-entry-box--compact__title a::attr(href)").extract()
        for page_link in page_links:
            yield scrapy.Request(url=page_link, callback=self.parse_article)

    def parse_article(self, response):
        page_url = response.url
        datetime = response.css("time.c-byline__item ::attr(datetime)").extract_first()
        if datetime is None:
            date = "There is no date here!"
        else:
            datetime = datetime.split('T')[0]
            datetime = datetime.split('-')
            date = [datetime[1], datetime[2], datetime[0]]

        yield {
            "links": page_url,
            "date": date
        }

In [ ]:
# To view all the crawled links

%cd /content/drive/MyDrive/Colab Notebooks/Link_Crawler

!scrapy crawl the_verge_scrape

In [ ]:
# For test and interaction

!scrapy shell "https://www.theverge.com/archives/tech/2020/2"

In [ ]:
# To append the crawled links to the Guardian links txt file

%cd /content/drive/MyDrive/Colab Notebooks/Link_Crawler

!scrapy crawl the_verge_scrape -o the_verge_scrape.csv

In [ ]:
import pandas as pd
# use pandas to input the keys and values on every iteration
df6 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Link_Crawler/the_verge_scrape.csv") 
df6.head()

,links
0,https://www.theverge.com/good-deals/2018/12/28...
1,https://www.theverge.com/2018/12/27/18156311/2...
2,https://www.theverge.com/2018/8/31/17805266/ap...
3,https://www.theverge.com/2018/8/30/17801018/yo...
4,https://www.theverge.com/circuitbreaker/2018/8...


In [ ]:
for index, row in df6.iterrows():
    if row['date'] == 'There is no date here!':
        df6.drop(index, inplace=True)

In [ ]:
# Remove duplicates
links = df6['links'].values.tolist()
# dates = df6['date'].values.tolist()

In [ ]:
print(f"The number of links is {len(links)}")
# print(f"The number of dates is {len(dates)}")

The number of links is 122


In [ ]:
!pip install newspaper3k

from newspaper import Article
import nltk
nltk.download('punkt')

In [ ]:
# build a function to convert keywords list to strings

seperator = ', '

def converttostr(input_seq, seperator):
    final_str = seperator.join(input_seq)
    return final_str

count = 1
# list_of_urls should contain two columns: The link, date-time
for ind, url in enumerate(links[:], start=1):
    article = Article(url, language="en")
    article.download() 
    article.parse() 
    article.nlp() 

    
    with open("Technology File2.txt", "a") as f:
        keywords = converttostr(article.keywords, seperator)
        f.write(f"Keywords: [{keywords}]" + "\n" + "\n")
        f.write(article.title + "\n")
        f.write(date + "\n" + "\n")
        f.write(article.text + "\n" + "\n")
        f.write(f"This is the article summary \n{article.summary}" + "\n" + "\n" + "\n" + "\n" + "\n")

    print(f"Finish with article {ind}")

Finish with article 1
Finish with article 2
Finish with article 3
Finish with article 4
Finish with article 5
Finish with article 6
Finish with article 7
Finish with article 8
Finish with article 9
Finish with article 10
Finish with article 11
Finish with article 12
Finish with article 13
Finish with article 14
Finish with article 15
Finish with article 16
Finish with article 17
Finish with article 18
Finish with article 19
Finish with article 20
Finish with article 21
Finish with article 22
Finish with article 23
Finish with article 24
Finish with article 25
Finish with article 26
Finish with article 27
Finish with article 28
Finish with article 29
Finish with article 30
Finish with article 31
Finish with article 32
Finish with article 33
Finish with article 34
Finish with article 35
Finish with article 36
Finish with article 37
Finish with article 38
Finish with article 39
Finish with article 40
Finish with article 41
Finish with article 42
Finish with article 43
Finish with article 